# AgentKit SDK Tutorial 02: 使用 AgentConfig 和 AgentKitClient

本教程可以**单独使用**，适合希望更优雅地管理配置和多次操作的用户。

在本 Notebook 中，你将学会：

1. 使用 `AgentConfig` 读取和修改 `agentkit.yaml` 配置
2. 使用 `AgentKitClient` 复用配置，多次执行 build / deploy / invoke 等操作
3. 可选：为 Client 配置 `LoggingReporter`，将进度输出到日志

> 提示：
> - Notebook 会自动在 `tutorial_projects/t02-client-config` 下创建一个 basic 示例项目（如不存在）；
> - 即使你没有跑过 Tutorial 01，本教程也可以直接从上到下运行。

## 0. 环境准备

在开始本教程之前，请确保你已经**正确配置了凭证**。AgentKit 需要两类凭证：

### 第一类：SDK 与 CLI 的云服务访问凭证
用于创建和管理云资源（CR、CP、TOS、Runtime 等）。推荐配置方式：

```bash
# 使用 agentkit 命令行工具全局配置（推荐）
agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
agentkit config --global --set volcengine.region=cn-beijing
```

### 第二类：Agent 运行时的环境变量
用于 Agent 访问大模型等服务。在 `agentkit.yaml` 的 `runtime_envs` 中配置(Cloud/Hybrid模式无需配置，Local模式需配置)：

```python
# 模型访问凭证（示例）
config.add_runtime_env("MODEL_AGENT_NAME", "your-model-name")
config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-api-key")
```

> **注意**：如果你还没有配置凭证，请先学习 `00_configuration_and_credentials.ipynb` 教程完成配置。

运行下面的单元，导入本教程需要的模块：
- 导入标准库 `pathlib.Path`
- 导入 Starter Toolkit SDK：`agentkit.toolkit import sdk`
- 导入 Client 与 Config：`AgentKitClient`, `AgentConfig`
- 导入预检枚举：`PreflightMode`
- 可选：导入 `LoggingReporter`，将进度信息输出到日志（更方便在 Notebook 中观察过程）


In [ ]:
from pathlib import Path
import logging

from agentkit.toolkit import sdk
from agentkit.toolkit.sdk import AgentKitClient, AgentConfig
from agentkit.toolkit.models import PreflightMode
from agentkit.toolkit.reporter import LoggingReporter

print("AgentKit SDK 相关模块已导入。")


## 1. 准备一个示例项目（如不存在则创建）

我们约定使用示例项目路径：`tutorial_projects/t02-client-config`。

- 如果该目录已经存在，则直接使用现有项目；
- 如果不存在，则通过 `sdk.init_project` 自动创建一个新的 basic 模板项目。


In [ ]:
project_root = Path("tutorial_projects/t02-client-config")
project_name = project_root.name

print(f'约定项目路径: {project_root.resolve()}')

if not project_root.exists():
    print("项目目录不存在，使用 sdk.init_project 创建示例项目...")
    init_result = sdk.init_project(
        project_name=project_name,
        template="basic",
        project_root=str(project_root),
    )
    print("初始化是否成功:", init_result.success)
    print("项目路径:", init_result.project_path)
else:
    print("项目目录已存在，直接复用。")

config_file = project_root / "agentkit.yaml"
print("配置文件路径:", config_file)

> **注意**：本教程假设你已经完成了凭证配置。如果尚未配置，请先运行 `00_configuration_and_credentials.ipynb` 教程。


## 2. 使用 AgentConfig 读取配置

`AgentConfig` 是一个 **SDK 友好的配置管理类**，它的作用是：

- 帮你加载 `agentkit.yaml`
- 提供属性访问（如 `config.launch_type`）和链式 API（如 `set_launch_type(...)`）
- 自动处理内部配置结构，不需要你手动解析 YAML

下面我们先加载刚刚的示例项目配置，并查看几个关键字段。


In [ ]:
# 使用 AgentConfig.load 加载配置（可以传目录或 agentkit.yaml 的路径）
config = AgentConfig.load(project_root)

print("AgentConfig 基本信息:")
print(config)  # 人类可读的字符串表示

print("常用字段示例:")
print("launch_type:", config.launch_type)
print("agent_name:", config.agent_name)
print("entry_point:", config.entry_point)
print("language:", config.language, config.language_version)
print("runtime_envs:", config.runtime_envs)
print("config", config.get_strategy_config("cloud"))


## 3. 修改配置并保存（AgentConfig 的常见用法）

你可以通过两种方式修改配置：

1. **属性方式**：直接给 `config.launch_type`、`config.description` 等赋值；
2. **链式 API**：使用 `set_launch_type` / `set_description` / `add_runtime_env` 等方法；

修改完成后，可以调用 `config.save()` 将变更写回文件。


In [ ]:
# 示例：修改描述和运行模式，并添加一个环境变量（仅示例，按需修改）
config.description = "A demo agent created in tutorials."

# 使用链式 API 修改 launch_type，并添加运行时环境变量
config.set_launch_type("local") \
              .add_runtime_env("HELLO", "WORLD")

# 保存修改（会写回 agentkit.yaml）
config.save()

print("修改后的配置概要:")
print(config)
print("runtime_envs:", config.runtime_envs)


### 3.1 配置特定 strategy（如 cloud）的参数

除了 `Common` 部分的基础字段（如 `launch_type`、`agent_name` 等），`agentkit.yaml` 中还包含按部署策略（strategy）划分的配置块，例如：

- `launch_types.local`: 本地 Docker 运行相关配置
- `launch_types.cloud`: 云上 Runtime 相关配置
- `launch_types.hybrid`: 本地构建 + 云上部署相关配置

在 SDK 中可以通过：

- `config.get_strategy_config("cloud")`: 读取某个 strategy 的配置（字典）
- `config.update_strategy_config({...}, strategy_name="cloud")`: 更新某个 strategy 的配置

下面我们用一个简单例子演示如何为 `cloud` strategy 设置 `region`、`runtime_name` 等字段。

In [ ]:
# 示例：为 cloud strategy 更新部分配置

# 如果你希望后续使用 cloud 模式，可以先把 launch_type 改为 "cloud"
config.set_launch_type("cloud").save()

print("当前 launch_type:", config.launch_type)

# 读取当前 cloud strategy 的配置（如果尚未配置，可能是一个空字典或包含默认字段）
cloud_cfg = config.get_strategy_config("cloud")
print("更新前 cloud strategy 配置:")
print(cloud_cfg)

# 使用 update_strategy_config 更新 cloud strategy 的部分字段
config.update_strategy_config(
    {
        "region": "cn-beijing",
        "runtime_name": "tutorial-runtime",
    },
    strategy_name="cloud",
)

# 再次读取并查看更新后的配置
updated_cloud_cfg = config.get_strategy_config("cloud")
print("更新后 cloud strategy 配置:")
print(updated_cloud_cfg)

## 4. 使用 AgentKitClient 复用配置

有了 `AgentConfig` 之后，就可以把它交给 `AgentKitClient` 来**复用配置**：

- 不需要在每次调用 `sdk.build/deploy/invoke` 时重复传 `config_file`
- 可以在 Client 层统一指定 `reporter`（例如 `LoggingReporter`），记录所有操作日志

下面我们：

1. 配置一个简单的 Python 日志记录器；
2. 使用 `AgentConfig` 实例构造 `AgentKitClient`；
3. 通过 Client 依次执行 build / deploy / invoke。


In [ ]:
# 配置一个简单的 logger，将信息输出到控制台
logger = logging.getLogger("agentkit_tutorial")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("[%(levelname)s] %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

reporter = LoggingReporter(logger=logger)

# 使用 AgentConfig 和自定义 Reporter 创建 Client
client = AgentKitClient(config=config, reporter=reporter)
print("已创建 AgentKitClient:", client)

# 通过 Client 进行构建（复用同一套配置，不必手动传 config_file）
build_result = client.build(
    platform="auto",
    preflight_mode=PreflightMode.WARN,
)
print("[Client.build] 成功:", build_result.success)
print("镜像信息 image:", build_result.image)


In [ ]:
# 通过 Client 进行部署
deploy_result = client.deploy(
    preflight_mode=PreflightMode.WARN,
)
print("[Client.deploy] 成功:", deploy_result.success)
print("endpoint_url:", deploy_result.endpoint_url)


In [ ]:
# 通过 Client 调用已部署的 Agent
invoke_result = client.invoke({"prompt": "请简单介绍一下 AgentKit SDK。"}, headers={"user_id": "user-1", "session_id": "session-1"})
print("[Client.invoke] 成功:", invoke_result.success)
print("响应内容:")
print(invoke_result.response)


## 5. 使用 Client 查询状态与清理

Client 同样封装了 `status` / `destroy` 接口，方便你在 Notebook 或服务代码中管理 Agent 生命周期。

> 注意：`destroy` 会销毁运行时资源，是一个相对“重”的操作，建议在确认不再需要该服务时再执行。

In [ ]:
# 查询当前服务状态
status_result = client.status()
print("[Client.status] 成功:", status_result.success)
print("当前状态 status:", status_result.status)
print("endpoint_url:", status_result.endpoint_url)


In [ ]:
# 可选：完全销毁服务及相关资源（**慎用**）
destroy_result = client.destroy()
print("[Client.destroy] 成功:", destroy_result.success, destroy_result.error)

## 6. 小结

在本教程中，你学习了：

1. 使用 `AgentConfig.load()` 加载和浏览 Agent 配置
2. 通过属性与链式 API 修改配置并保存到 `agentkit.yaml`
3. 使用 `AgentKitClient` 复用配置，多次执行 build / deploy / invoke / status 等操作
4. 为 Client 配置 `LoggingReporter`，将进度信息输出到日志

下一步推荐：

- 根据你自己的业务需求，修改模板生成的 Agent 代码和配置；
- 尝试在 Notebook 或后端服务中集成 `AgentKitClient`，构建更复杂的自动化工作流；
- 阅读更多文档，了解 `launch_type`、本地/云端/混合部署策略的详细行为。
